In [10]:
import sys
import jieba
import string
import pandas as pd
from IPython.display import display
import chardet

In [11]:
#setting the output size of print dataframe
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.width',180)

In [12]:
def Cutting(text):
  #mix the number without comma
  punct=string.punctuation
  text=text.replace(',','')
  text=text.replace(' ','')
  #load the dictionary
  jieba.load_userdict('dict.txt')

  #cutting
  Cut_Result=jieba.lcut(text)

  return Cut_Result


In [13]:
def Check_Index(Cut,Tar):
  Tar_Index=[]
  Index=0
  #Last=Cut.rindex(Tar)
  #while Index!=Last:
  #print(Cut)

  '''while(1):
    try:
      Index=Cut.index(Tar,Index+1)
      Tar_Index.append(Index)
    except ValueError:
      break
  '''
  for i in range(len(Cut)):
    if Tar in Cut[i]:
      Tar_Index.append(i)
  
  return Tar_Index

In [14]:
def GetTrueNum():
  TrueNum=1
  return TrueNum

In [15]:

def Check_Num(target,numlist,truenum):
  #target is the name of event and numlist is the list include all the number near the target
  correctness="False"
  #tar_num=get_tar_num要改成項目掛勾之正確數字
  #turenum=626479
  for num in numlist:
    if int(num)==int(truenum):
      correctness="True"
    #只要有一項正確就會改成True  
  return correctness

In [16]:
def GetNumStr(numlist):
  numstring=""
  #delete the duplicated number
  #numlist=[]
  #for num in dnumlist:
  #  if num not in numlist:
  #    numlist.append(num)
  #convert
  numlist=list(set(numlist))
  for i in range(len(numlist)):
    numstring+=numlist[i]
    if (i==len(numlist)-1):
      return numstring
    else:
      numstring+=","

In [17]:
def Get_Num(Cut,Index,Target,Truenum):
  result=[]
  for i in Index:
    Indexint=int(i)
    event=[]
    event.append(Target)
    numlist=[]
    num=[]
    for j in range(Indexint,Indexint+10):
      if(all(elem.isdigit()for elem in Cut[j])):
        num.append(str(Cut[j]))
    #append the number list, Remark, and the check result
    #for number in numlist:
    #  if number not in numlist:
    #    num.append(number)
    if(len(num)==0):
      #event.append("NULL")
      event.append("NULL")
      event.append("沒有數字項在項目附近")
      event.append("無須勾稽")
    elif(len(num)==1):
    #if numlist isn't empty check the correctness of the number within the database
      #event.append(num)
      event.append(GetNumStr(num))
      event.append("-")
      event.append(Check_Num(Target,num,Truenum))
    else:
      #event.append(num)
      event.append(GetNumStr(num))
      event.append("附近多於一個數")
      event.append(Check_Num(Target,num,Truenum))

    #check the correctness of the number within the database

    '''if (len(num)>1):
      event.append("多於一個數")
    elif (len(num)==1):
      event.append("-")
    else :
      event.append("沒有數字項在項目附近")'''
    result.append(event)
  return result



# 測試單項檢核項目

In [18]:
text="民國110年度餐飲收入及客房收入分別為新台幣1,625,601仟元及新台幣收入626,479仟元，分別佔合併總營業收入收入69.65%及26.84%。有關收入1625601營業收入之收入會1625601計政策，請詳合併財務報表附註四1625601（三十）；會計項目說明，收入請詳合併財務報表附註六（二十三）。"
Cut_Result=Cutting(text)
#print(Cut_Result) #check cut result

#load in the target string
Target='收入'
truenum="626479"
#print(Cut_Result.index(Target))
#check the all Index of Target string
Tar_Index=Check_Index(Cut_Result,Target)
#print(Tar_Index)

#check the number around the index
near_num=Get_Num(Cut_Result,Tar_Index,Target,truenum)
#print(near_num)


#convert into dataframe
df=pd.DataFrame(near_num,columns=["項目","數字","備註","勾稽結果"])
#print(df)
display(df)

,項目,數字,備註,勾稽結果
0,收入,1625601,-,False
1,收入,"626479,1625601",附近多於一個數,True
2,收入,626479,-,True
3,收入,1625601,-,False
4,收入,1625601,-,False
5,收入,1625601,附近多於一個數,False
6,收入,1625601,-,False
7,收入,1625601,-,False
8,收入,NULL,沒有數字項在項目附近,無須勾稽


# 測試多項檢核項目

In [19]:
#try multiple Target
text="民國110年度營業收入餐飲收入626479及客房收入分別為新台幣1,625,601仟元及新台幣收入626,479仟元，分別佔合併總營業收入收入69.65%及26.84%。有關收入1625601營業收入之收入會1625601計政策，請詳合併626479財務報表附註四1625601（三十）；會計項目說明，收入請詳合併財務報表附註六（二十三）。"
Cut_Result=Cutting(text)

Target=["收入","新台幣","新臺幣","營業收入","會計"]#,"新台幣","新臺幣","營業收入"
truenumlist=["626479","1625601","626479","626479","123456"]
result_table=[]

for i in range(len(Target)):
  tar=Target[i]
  truenum=truenumlist[i]
  Tar_Index=Check_Index(Cut_Result,tar)
  near_num=Get_Num(Cut_Result,Tar_Index,tar,truenum)
  for i in near_num:
    result_table.append(i)
df=pd.DataFrame(result_table,columns=["項目","數字","備註","勾稽結果"])
#df=pd.DataFrame(result_table)
display(df)


,項目,數字,備註,勾稽結果
0,收入,626479,-,True
1,收入,"626479,1625601",附近多於一個數,True
2,收入,"626479,1625601",附近多於一個數,True
3,收入,626479,-,True
4,收入,1625601,-,False
5,收入,1625601,-,False
6,收入,1625601,附近多於一個數,False
7,收入,1625601,-,False
8,收入,"626479,1625601",附近多於一個數,True
9,收入,NULL,沒有數字項在項目附近,無須勾稽


In [20]:
path='2707_2019_Q4.txt'
f = open(path,'rb')
r = f.read()#獲取文本的編碼方式
f_charInfo = chardet.detect(r)
print(f_charInfo) #輸出文本格式信息
print(f_charInfo['encoding']) #取得文本格式
text=r.decode(f_charInfo['encoding'])
print(r.decode(f_charInfo['encoding'])) #通過取得的文本格式讀取txt
f.close()

Cut_Result=Cutting(text)

print(Cut_Result)

Target=["使用權資產","租賃負債","資產總額","營業收入"]#,"新台幣","新臺幣","營業收入"
truenumlist=["3819177","3751080","11332199","6535609"]
result_table=[]

for i in range(len(Target)-2):
  tar=Target[i]
  truenum=truenumlist[i]
  Tar_Index=Check_Index(Cut_Result,tar)
  near_num=Get_Num(Cut_Result,Tar_Index,tar,truenum)
  for i in near_num:
    result_table.append(i)
df=pd.DataFrame(result_table,columns=["項目","數字","備註","勾稽結果"])
#df=pd.DataFrame(result_table)
display(df)

FileNotFoundError: [Errno 2] No such file or directory: '2707_2019_Q4.txt'

# 以下是測試用code不用跑




In [ ]:
#text="  \nD.履約保證金：故宮晶華提供履約保證函$5,000，於依約完成所有\n資產移轉之相關程序後 3 個月內返還之。 \n(3)限制條款： \nA.開發經營期間應維持下列財務比率： \n  a.流動比率不得低於 100%。 \n  b.總負債金額不得超過淨值 1.5 倍。 \nB.故宮晶華之財務計劃應以提出經甄選評決之財務計劃為調整基\n礎。 \nC.除經國立故宮博物院同意，故宮晶華不得轉投資其他事業。 \nD.故宮晶華因經營本計劃所取得之資產及負債，不得設定任何負擔；\n非經國立故宮博物院同意，不得轉讓、出租。 \n9.達美樂為取得「DOMINO'S PIZZA」之營業權，於民國 107 年 1 月 1 日與\nDomino's Pizza International Franchising Inc.(DPIF)簽訂合約，\n其主要內容如下： \n(1)合約期限：至民國 121 年 12 月 31 日止。 \n(2)權利金： \n   A.開店權利金： \n (A)民國 107 年度為每店美金 3,000 元。 \n (B)民國 108 年度為每店美金 4,000 元。 \n     (C)民國 109 年度起為每店美金 5,000 元。  \n   B.營業權利金：按每期營業收入 3%收取，繳交予 DPIF。 \n10.營業租賃協議 \n請詳附註六(九)、(十)、(十八)及(三十五)說明。 \n11.子公司-FIH Management Limited(以下簡稱「FML」)於民國 107 年 7 月\n1 日與 IHG 及 RHW 簽訂股東協議合約，內容包括 RHW 未來之內部管理\n運作、相關成本費用分攤、Regent 品牌發展規劃及使用限制、財務審\n計資訊及股權買賣之相關事項等，其中並約定股東雙方於民國 115 年\n至民國 122 年間對 FML 所持有之 RHW 49%股權，FML 有向 IHG 出售之權\n利，IHG 亦有向 FML 購買之權利，交易價格以「行使權利之前一年度依\n合約約定調節後營業收入之 11 倍」計算。 \n十、重大之災害損失 \n無此情形。 \n十一、重大之期後事項 \n本公司於民國 109 年 3 月 23 日經董事會提議民國 108 年度盈餘分派案，請\n詳附註六(二十六)4.之說明。 \n十二、其他 \n(一)資本管理 \n本集團之資本管理目標，係為保障集團能繼續經營，維持最佳資本結構\n以降低資金成本，並為股東提供報酬。為了維持或調整資本結構，本集\n團可能會調整支付予股東之股利金額、退還資本予股東、發行新股或出\n售資產以降低債務。本集團利用負債比率以監控其資本，該比率係按總\n負債除以總權益計算。 \n~64~ "
punct=string.punctuation
#for i in punct:
  #text=text.replace(i,"")
text=text.replace(',',"")
text=text.replace(' ','')
###text=text.replace(" ","")
print(text)
print('|'.join(jieba.cut(text,cut_all=False,HMM=True)))
print('|'.join(jieba.cut(text,cut_all=False,HMM=False)))
print('|'.join(jieba.cut(text,cut_all=True,HMM=True)))
print('|'.join(jieba.cut(text,cut_all=True,HMM=False)))

晶華國際酒店股份有限公司公鑒：查核意見晶華國際酒店股份有限公司及子公司（以下簡稱「晶華集團」）民國108年及107年12月31日之合併資產負債表，暨民國108年及107年1月1日至12月31日之合併綜合損益表、合併權益變動表、合併現金流量表，以及合併財務報表附註(包括重大會計政策彙總)，業經本會計師查核竣事。依本會計師之意見，基於本會計師之查核結果及其他會計師之查核報告（請參閱其他事項段），上開合併財務報表在所有重大方面係依照「證券發行人財務報告編製準則」暨金融監督管理委員會認可之國際財務報導準則、國際會計準則、解釋及解釋公告編製，足以允當表達晶華集團民國108年及107年12月31日之合併財務狀況，暨民國108年及107年1月1日至12月31日之合併財務績效及合併現金流量。查核意見之基礎本會計師係依照「會計師查核簽證財務報表規則」及中華民國一般公認審計準則執行查核工作。本會計師於該等準則下之責任將於會計師查核合併財務報表之責任段進一步說明。本會計師所隸屬事務所受獨立性規範之人員已依中華民國會計師職業道德規範，與晶華集團保持超然獨立，並履行該規範之其他責任。基於本會計師之查核結果及其他會計師之查核報告，本會計師相信已取得足夠及適切之查核證據，以作為表示查核意見之基礎。關鍵查核事項關鍵查核事項係指依本會計師之專業判斷，對晶華集團民國108年度合併財務報表之查核最為重要之事項。該等事項已於查核合併財務報表整體及形成查核意見之過程中予以因應，本會計師並不對該等事項單獨表示意見。晶華集團民國108年度合併財務報表之關鍵查核事項如下：餐旅服務收入認列之正確性事項說明有關收入認列之會計政策，請詳合併財務報告附註四(三十五)；營業收入會計科目說明，請詳合併財務報告附註六(二十七)。晶華集團提供客房住宿、餐飲服務、租賃及經營管理技術服務等，其中以餐旅服務收入為主要營業項目，由於金額重大且因行業特性之故，產品單價低而銷售筆數眾多，交易量龐大，錯誤發生之可能性相對較高，可能導致合併財務報表重大不實表達，因此本會計師將餐旅服務收入認列之正確性列為查核重要事項之一。因應之查核程序本會計師對上開關鍵查核事項已執行之因應程序彙列如下:1.瞭解及測試晶華集團所採用餐旅服務收入內部控制之有效性，包括確認銷售系統產生之銷售報表金額與入帳金額一致。2.進行旅館及餐廳各式管理報表分析，包括分析住房率及房

In [ ]:
print(jieba.lcut(text))

['民國', '110', '年度', '餐飲', '收入', '及', '客房', '收入', '分別', '為', '新台幣', '1625601', '仟元及', '(', '透過', '損益', '按', '公允', '價值', '衡量', '之', '金融', '資產', '－', '流動', ')', '新台幣', '626479', '仟元', '，', '分別', '佔', '合併', '總', '營業收入', '69.65%', '及', '26.84%', '。', '有關', '營業收入', '之會計', '政策', '，', '請詳', '合併', '財務', '報表', '附', '註', '四', '（', '三十', '）', '；', '會計', '項目', '說明', '，', '請詳', '合併', '財務', '報表', '附', '註', '六', '（', '二十三', '）', '。']


In [ ]:
jieba.load_userdict('test.txt')
result=jieba.lcut(text)
print(result)
print(result[5])

['民國', '110', '年度', '餐飲', '收入', '及', '客房', '收入', '分別', '為', '新台幣', '1625601', '仟元及', '(', '透過', '損益', '按', '公允', '價值', '衡量', '之', '金融', '資產', '－', '流動', ')', '新台幣', '626479', '仟元', '，', '分別', '佔', '合併', '總', '營業收入', '69.65%', '及', '26.84%', '。', '有關', '營業收入', '之會計', '政策', '，', '請詳', '合併', '財務', '報表', '附', '註', '四', '（', '三十', '）', '；', '會計', '項目', '說明', '，', '請詳', '合併', '財務', '報表', '附', '註', '六', '（', '二十三', '）', '。']
及


In [ ]:
target="營業收入"
target_index=result.index(target)
print("target位置",target_index)
for i in range(target_index,target_index+10):
  print("target後",i-target_index,"位：",result[i])
print(result[target_index+8])
IsNum=all(elem.isdigit()for elem in result[target_index+1])
print(IsNum)

target位置 34
target後 0 位： 營業收入
target後 1 位： 69.65%
target後 2 位： 及
target後 3 位： 26.84%
target後 4 位： 。
target後 5 位： 有關
target後 6 位： 營業收入
target後 7 位： 之會計
target後 8 位： 政策
target後 9 位： ，
政策
False


In [ ]:
import MySQLdb
import pandas as pd
conn = MySQLdb.Connect(host = 'localhost',
                       port = 3307,
                       user = 'root',
                       db = 'kpmg',
                       charset='utf8')
cur = conn.cursor()

# company_id_input = input("輸入公司id : ")
# kind_input = input("輸入kind : ")
# year_input = input("輸入year : ")
# season_input = input("輸入season : ")

company_id_input = 2707
kind_input = "Balance_Sheet"
year_input = 2019
season_input = 4
流動資產合計 = "現金及約當現金+透過損益按公允價值衡量之金融資產－流動+按攤銷後成本衡量之金融資產－流動+應收票據淨額+應收帳款淨額+應收帳款－關係人淨額+其他應收款+其他應收款－關係人+本期所得稅資產+存貨+預付款項+其他流動資產+其他金融資產－流動+其他流動資產－其他"
cur.execute(f"SELECT `path` FROM `financial_statements` WHERE `company_id` = {company_id_input} AND `kind` = '{kind_input}' AND `year` = {year_input} AND `season` = {season_input} ")
result_set = cur.fetchall()

df = pd.read_csv(f"../../{result_set[0][0]}")

In [ ]:
流動資產合計 = {"流動資產合計":"現金及約當現金+透過損益按公允價值衡量之金融資產－流動+按攤銷後成本衡量之金融資產－流動+應收票據淨額+應收帳款淨額+應收帳款－關係人淨額+其他應收款+其他應收款－關係人+本期所得稅資產+存貨+預付款項+其他流動資產+其他金融資產－流動+其他流動資產－其他"}
all_function = [流動資產合計]
new_dict = {}
kind_list = []
money_list = []
status_list = []
for f in all_function:
    for k ,v in f.items():
        temp_sum = 0
        temp_ls = v.split("+")
        for i in temp_ls:
            filt = (df["會計項目Accounting Title"].str.contains(i))
            x = df.loc[filt]
            # print(i)
            try:
                temp_num = x.iloc[0,[3]][0]
                # print(temp_num)
                if "(" in temp_num:
                    temp_num = temp_num.replace("()","")
                    temp_num = -(int(temp_num))
                    temp_sum+=temp_num
                    pass
                else:
                    temp_num = int(temp_num)
                    temp_sum += temp_num 
            except:
                pass

        k_filt = (df["會計項目Accounting Title"].str.contains(k))
        k_x = df.loc[k_filt]
        k_num = k_x.iloc[0,[3]][0]
        if "(" in k_num:
            k_num = temp_num.replace("()","")
            k_num = -(int(k_num))
            pass
        else:
            k_num = int(k_num)

        if k_num == temp_sum:
            kind_list.append(k)
            money_list.append(k_num)
            status_list.append("True")
        else:
            kind_list.append(k)
            money_list.append(k_num)
            status_list.append("False")

        new_dict["會科"]=kind_list
        new_dict["金額"]=money_list
        new_dict["符合"]=status_list

df = pd.DataFrame(new_dict)
df

NameError: ignored